<a href="https://colab.research.google.com/github/marcelounb/Deep_Learning_with_python_JasonBrownlee/blob/master/12_3_Lift_Performance_By_Standardizing_Boston_House_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lift Performance By Standardizing The Dataset

An important concern with the Boston house price dataset is that the input attributes all vary in their scales because they measure di↵erent quantities. It is almost always good practice to prepare your data before modeling it using a neural network model. Continuing on from the above baseline model, we can re-evaluate the same model using a standardized version of the input dataset. 

We can use scikit-learn’s Pipeline framework to perform the standardization during the model evaluation process, within each fold of the cross validation. This ensures that there is no data leakage from each testset cross validation fold into the training data. The code below creates a scikit-learn Pipeline that ﬁrst standardizes the dataset then creates and evaluates the baseline neural network model


In [1]:
import numpy as np 
import pandas as pd 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
# load dataset 
dataframe = pd.read_csv("/content/housing.csv", delim_whitespace=True, header=None) 
dataset = dataframe.values 
# split into input (X) and output (Y) variables 
X = dataset[:,0:13] 
Y = dataset[:,13]

In [3]:
X[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [4]:
print(X.shape, Y.shape)

(506, 13) (506,)


# Develop a Baseline Neural Network Model

We can create Keras models and evaluate them with scikit-learn by using handy wrapper objects provided by the Keras library. This is desirable, because scikit-learn excels at evaluating models and will allow us to use powerful data preparation and model evaluation schemes with very few lines of code. 

The Keras wrapper class require a **function** as an argument. This function that we must deﬁne is responsible for creating the neural network model to be evaluated. 

Below we **deﬁne the function to create the baseline model to be evaluated.** It is a simple model that has a single fully connected hidden layer with the same number of neurons as input attributes (13). 

The network uses good practices such as the **rectiﬁer** activation function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform. 

The efficient ADAM optimization algorithm is used and a mean squared error loss function is optimized. This will be the same metric that we will use to evaluate the performance of the model. It is a desirable metric because by taking the square root of an error value it gives us a result that we can directly understand in the context of the problem with the units in thousands of dollars.


In [0]:
# define base mode 
def baseline_model(): 
  # create model 
  model = Sequential() 
  model.add(Dense(13, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' )) 
  model.add(Dense(1, init= 'normal' )) 
  # Compile model 
  model.compile(loss= 'mean_squared_error' , optimizer= 'adam' ) 
  return model

The Keras wrapper object for use in scikit-learn as a regression estimator is called KerasRegressor. We create an instance and pass it both the name of the function to create the neural network model as well as some parameters to pass along to the fit() function of the model later, such as the number of epochs and batch size. Both of these are set to sensible defaults. 

We also initialize the random number generator with a constant random seed, a process we will repeat for each model evaluated in this tutorial. This is to ensure we compare models consistently and that the results are reproducible.


In [0]:
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)
# evaluate model with standardized dataset 
estimators = []
estimators.append(( 'standardize' , StandardScaler())) 
estimators.append(( 'mlp' , KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5)))  # ,verbose=0
pipeline = Pipeline(estimators)

The ﬁnal step is to evaluate this baseline model. We will use 10-fold cross validation to evaluate the model.

In [8]:
kfold = KFold(n_splits=10, random_state=seed) 
results = cross_val_score(pipeline, X, Y, cv=kfold) 
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  


Epoch 1/100
455/455 [==============================] - 0s 335us/step - loss: 598.5821
Epoch 2/100
455/455 [==============================] - 0s 179us/step - loss: 553.8186
Epoch 3/100
455/455 [==============================] - 0s 189us/step - loss: 461.5912
Epoch 4/100
455/455 [==============================] - 0s 198us/step - loss: 337.9341
Epoch 5/100
455/455 [==============================] - 0s 185us/step - loss: 224.5910
Epoch 6/100
455/455 [==============================] - 0s 183us/step - loss: 142.9415
Epoch 7/100
455/455 [==============================] - 0s 203us/step - loss: 95.6328
Epoch 8/100
455/455 [==============================] - 0s 193us/step - loss: 70.2092
Epoch 9/100
455/455 [==============================] - 0s 188us/step - loss: 55.2281
Epoch 10/100
455/455 [==============================] - 0s 212us/step - loss: 45.7332
Epoch 11/100
455/455 [==============================] - 0s 220us/step - loss: 39.4513
Epoch 12/100
455/455 [==============================] - 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  


Epoch 1/100
455/455 [==============================] - 0s 282us/step - loss: 584.3774
Epoch 2/100
455/455 [==============================] - 0s 186us/step - loss: 554.7180
Epoch 3/100
455/455 [==============================] - 0s 201us/step - loss: 467.2371
Epoch 4/100
455/455 [==============================] - 0s 208us/step - loss: 336.2641
Epoch 5/100
455/455 [==============================] - 0s 199us/step - loss: 219.9553
Epoch 6/100
455/455 [==============================] - 0s 183us/step - loss: 145.8016
Epoch 7/100
455/455 [==============================] - 0s 179us/step - loss: 102.7452
Epoch 8/100
455/455 [==============================] - 0s 179us/step - loss: 75.6434
Epoch 9/100
455/455 [==============================] - 0s 180us/step - loss: 58.6363
Epoch 10/100
455/455 [==============================] - 0s 179us/step - loss: 48.0530
Epoch 11/100
455/455 [==============================] - 0s 194us/step - loss: 41.7818
Epoch 12/100
455/455 [==============================] - 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  


...

Epoch 1/100
456/456 [==============================] - 0s 294us/step - loss: 596.9534
Epoch 2/100
456/456 [==============================] - 0s 177us/step - loss: 548.1101
Epoch 3/100
456/456 [==============================] - 0s 182us/step - loss: 442.0297
Epoch 4/100
456/456 [==============================] - 0s 196us/step - loss: 303.5503
Epoch 5/100
456/456 [==============================] - 0s 185us/step - loss: 186.7181
Epoch 6/100
456/456 [==============================] - 0s 181us/step - loss: 116.3040
Epoch 7/100
456/456 [==============================] - 0s 179us/step - loss: 77.3397
Epoch 8/100
456/456 [==============================] - 0s 183us/step - loss: 55.2156
Epoch 9/100
456/456 [==============================] - 0s 198us/step - loss: 42.6588
Epoch 10/100
456/456 [==============================] - 0s 190us/step - loss: 35.1597
Epoch 11/100
456/456 [==============================] - 0s 179us/step - loss: 30.9952
Epoch 12/100
456/456 [==============================] - 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  


Epoch 1/100
456/456 [==============================] - 0s 296us/step - loss: 625.8937
Epoch 2/100
456/456 [==============================] - 0s 208us/step - loss: 582.4883
Epoch 3/100
456/456 [==============================] - 0s 179us/step - loss: 493.3987
Epoch 4/100
456/456 [==============================] - 0s 182us/step - loss: 379.7158
Epoch 5/100
456/456 [==============================] - 0s 179us/step - loss: 267.3797
Epoch 6/100
456/456 [==============================] - 0s 206us/step - loss: 174.7651
Epoch 7/100
456/456 [==============================] - 0s 205us/step - loss: 115.6910
Epoch 8/100
456/456 [==============================] - 0s 218us/step - loss: 82.3935
Epoch 9/100
456/456 [==============================] - 0s 206us/step - loss: 63.6142
Epoch 10/100
456/456 [==============================] - 0s 267us/step - loss: 51.8729
Epoch 11/100
456/456 [==============================] - 0s 204us/step - loss: 44.4288
Epoch 12/100
456/456 [==============================] - 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  


Epoch 1/100
456/456 [==============================] - 0s 313us/step - loss: 608.6856
Epoch 2/100
456/456 [==============================] - 0s 199us/step - loss: 564.7138
Epoch 3/100
456/456 [==============================] - 0s 204us/step - loss: 472.9639
Epoch 4/100
456/456 [==============================] - 0s 196us/step - loss: 350.1657
Epoch 5/100
456/456 [==============================] - 0s 195us/step - loss: 232.8251
Epoch 6/100
456/456 [==============================] - 0s 196us/step - loss: 148.4961
Epoch 7/100
456/456 [==============================] - 0s 209us/step - loss: 98.1728
Epoch 8/100
456/456 [==============================] - 0s 195us/step - loss: 70.2017
Epoch 9/100
456/456 [==============================] - 0s 201us/step - loss: 53.8542
Epoch 10/100
456/456 [==============================] - 0s 216us/step - loss: 43.7997
Epoch 11/100
456/456 [==============================] - 0s 226us/step - loss: 37.4976
Epoch 12/100
456/456 [==============================] - 0

Running the example provides an improved performance over the baseline model without standardized data, dropping the error by 10 thousand squared dollars.